# Navigation of a "collect ripe bananas" environment


### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import random
import torch
from collections import deque
from dqn_agent import Agent
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:

env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


**_Brains_** are responsible for deciding the actions. Here we check for the first brain available, and set it as the default brain we will be using.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. DQN base model

This function will run or train the agent. If the agent will reach average score of 13.0 over 100 consecutive episodes it will save the learned weights to the model.pth file. You can read this weights while initializing new agent to run trained agent

In [6]:
agent = Agent(state_size=37, action_size=4, seed=0)

In [ ]:
def dqn(brain_name, n_episodes=1800, max_t=10000, eps_start=0.98, eps_end=0.01, eps_decay=0.90):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """

    # init the list that will track scores, the scores window which will track the last 100 scores
    # and init the epsilon used to determine randomness
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    for i_episode in range(1, n_episodes+1):
        # reset the environment before each episode
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        # init score to store score after each timestep
        score = 0
        for t in range(max_t):
            action = agent.act(state, 0)
            
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            
            #next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        # save most recent score
        scores_window.append(score)
        scores.append(score)
        # Let epsilon decay
        eps = max(eps_end, eps_decay*eps)
        
        # print progress
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=13:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            # save model when problem is solved
            torch.save(agent.qnetwork_local.state_dict(), 'dqn_model.pth')
            break
    return scores

scores = dqn(brain_name)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()


In [ ]:
#Testing the model
from dqn_agent import Agent
filename_to_load = './dqn_model.pth'
final_eps = 0.01

agent = Agent(state_size=37, action_size=4, seed=0)
agent.qnetwork_local.load_state_dict(torch.load(filename_to_load, map_location=lambda storage, loc: storage))
num_episodes = 100
scores = []
for i_episode in range(1,num_episodes+1):
    env_info = env.reset(train_mode=False)[brain_name] 
    state = env_info.vector_observations[0]            
    score = 0                                         
    while True:
        action = agent.act(state, eps=final_eps)
        action = np.int32(action)
        env_info = env.step(action)[brain_name]        
        next_state = env_info.vector_observations[0]   
        reward = env_info.rewards[0]                   
        done = env_info.local_done[0]

        score += reward                                
        state = next_state                             
        if done:                                       
            print('Episode {}: {}'. format(i_episode, score))
            scores.append(score)
            break

print('All the scores{}'.format(scores))
print("Mean Score: {}".format(np.mean(scores)))

Episode 1: 20.0
Episode 2: 20.0
Episode 3: 11.0


When finished, close the environment.

In [ ]:
env.close()